In [8]:
import math

def calculate_cpmm_purchase(pool, p, bet_amount, is_yes):
    # 1. New probability after bet
    new_prob = get_cpmm_probability_after_bet(pool, p, bet_amount, is_yes)
    
    # 2. Fees
    fee_rate = 0.02  # Assuming a 2% fee
    fees = bet_amount * fee_rate
    
    # 3. Log-odds (elasticity)
    log_odds = math.log(p / (1 - p))
    
    # 4. Change to the liquidity pool
    new_pool = pool + bet_amount - fees
    
    return {
        "new_probability": new_prob,
        "fees": fees,
        "log_odds": log_odds,
        "new_pool": new_pool
    }

def get_cpmm_probability_after_bet(pool, p, bet_amount, is_yes):
    if is_yes:
        return (pool * p + bet_amount) / (pool + bet_amount)
    else:
        return (pool * p) / (pool + bet_amount)

def get_cpmm_probability(pool, p):
    return p

# Example usage
initial_pool = 1000
initial_probability = 0.55
bet_amount = 50
is_yes_bet = True

result = calculate_cpmm_purchase(initial_pool, initial_probability, bet_amount, is_yes_bet)
print(result)


{'new_probability': 0.5714285714285714, 'fees': 1.0, 'log_odds': 0.2006706954621514, 'new_pool': 1049.0}


In [21]:
import math

def calculate_fees(bet_amount, pool):
    fee_fraction = 0.005
    max_fee = 5
    fee = min(bet_amount * fee_fraction, max_fee)
    if pool < 500:
        fee *= pool / 500
    return fee

def get_cpmm_probability(pool, p):
    return p * pool['NO'] / (p * pool['NO'] + (1 - p) * pool['YES'])

def calculate_cpmm_purchase(pool, p, bet_amount, is_yes):
    prob_before = get_cpmm_probability(pool, p)
    
    fees = calculate_fees(bet_amount, pool['YES'] + pool['NO'])
    adjusted_bet = bet_amount - fees
    
    if is_yes:
        new_pool = {
            'YES': pool['YES'] + adjusted_bet,
            'NO': pool['NO']
        }
    else:
        new_pool = {
            'YES': pool['YES'],
            'NO': pool['NO'] + adjusted_bet
        }
    
    new_prob = get_cpmm_probability(new_pool, p)
    
    shares_purchased = adjusted_bet / (new_prob if is_yes else (1 - new_prob))
    
    max_payout = bet_amount / new_prob - bet_amount if is_yes else bet_amount / (1 - new_prob) - bet_amount
    
    return {
        "probability_before": prob_before,
        "probability_after": new_prob,
        "shares_purchased": shares_purchased,
        "fees": fees,
        "new_pool": new_pool,
        "max_payout": max_payout
    }

# Example usage
pool = {'YES': 910, 'NO': 1099}
p = 0.55
bet_amount = 50
is_yes = True

result = calculate_cpmm_purchase(pool, p, bet_amount, is_yes)
print(result)

{'probability_before': 0.5961339316534346, 'probability_after': 0.5832559373756137, 'shares_purchased': 85.2970313921747, 'fees': 0.25, 'new_pool': {'YES': 959.75, 'NO': 1099}, 'max_payout': 35.725659690627836}


In [21]:
import math

def calculate_fees(bet_amount, pool):
    fee_fraction = 0.05
    return bet_amount * fee_fraction

def calculate_cpmm(p, yes_shares, no_shares, bet_amount, is_yes):
    prob_before = p
    fees = calculate_fees(bet_amount, yes_shares + no_shares)
    adjusted_bet = bet_amount - fees
    
    if is_yes:
        new_yes_shares = yes_shares + adjusted_bet
        new_no_shares = no_shares
    else:
        new_yes_shares = yes_shares
        new_no_shares = no_shares + adjusted_bet
    
    new_pool = new_yes_shares + new_no_shares
    new_p = new_yes_shares / new_pool
    
    shares_purchased = adjusted_bet / (new_p if is_yes else (1 - new_p))
    
    max_payout = bet_amount / new_p - bet_amount if is_yes else bet_amount / (1 - new_p) - bet_amount
    
    return {
        "probability_before": prob_before,
        "probability_after": new_p,
        "shares_purchased": shares_purchased,
        "fees": fees,
        "new_pool": new_pool,
        "max_payout": max_payout
    }

# Example usage
yes_shares = 910
no_shares = 1099
bet_amount = 50
p = 0.55
is_yes = True

result = calculate_cpmm(p, yes_shares, no_shares, bet_amount, is_yes)
print(result)

{'probability_before': 0.55, 'probability_after': 0.4655968879163628, 'shares_purchased': 102.0195822454308, 'fees': 2.5, 'new_pool': 2056.5, 'max_payout': 57.38903394255874}


In [18]:
import math

class CPMMMarket:
    def __init__(self, initial_liquidity, initial_probability):
        self.p = initial_probability
        self.pool = {
            'YES': initial_liquidity * initial_probability,
            'NO': initial_liquidity * (1 - initial_probability)
        }
        self.collected_fees = {'creator_fee': 0, 'platform_fee': 0, 'liquidity_fee': 0}

    def get_probability(self):
        return (self.p * self.pool['NO']) / ((1 - self.p) * self.pool['YES'] + self.p * self.pool['NO'])

    def calculate_fee(self, shares, prob):
        fee_constant = 0.07
        return fee_constant * prob * (1 - prob) * shares

    def get_fees_split(self, total_fees):
        creator_fee = total_fees * 0.5
        platform_fee = total_fees * 0.5
        return {'creator_fee': creator_fee, 'platform_fee': platform_fee, 'liquidity_fee': 0}

    def calculate_shares(self, bet_amount, outcome):
        y, n = self.pool['YES'], self.pool['NO']
        k = y ** self.p * n ** (1 - self.p)
        
        if outcome == 'YES':
            return y + bet_amount - (k * (bet_amount + n) ** (self.p - 1)) ** (1 / self.p)
        else:
            return n + bet_amount - (k * (bet_amount + y) ** -self.p) ** (1 / (1 - self.p))

    def place_bet(self, bet_amount, outcome):
        prob_before = self.get_probability()
        
        shares = self.calculate_shares(bet_amount, outcome)
        avg_prob = bet_amount / shares
        fee = self.calculate_fee(shares, avg_prob)
        
        remaining_bet = bet_amount - fee
        actual_shares = self.calculate_shares(remaining_bet, outcome)
        
        fees_split = self.get_fees_split(fee)
        for fee_type, amount in fees_split.items():
            self.collected_fees[fee_type] += amount
        
        if outcome == 'YES':
            self.pool['YES'] -= actual_shares - remaining_bet
            self.pool['NO'] += remaining_bet
        else:
            self.pool['YES'] += remaining_bet
            self.pool['NO'] -= actual_shares - remaining_bet
        
        prob_after = self.get_probability()
        
        return {
            'shares': actual_shares,
            'fees': fee,
            'probability_before': prob_before,
            'probability_after': prob_after
        }

# Example usage
market = CPMMMarket(initial_liquidity=1000, initial_probability=0.55)
result = market.place_bet(bet_amount=50, outcome='YES')
print(result)


{'shares': 92.38018434644835, 'fees': 1.6661210506581903, 'probability_before': 0.5, 'probability_after': 0.5462414423920766}


In [26]:
import math

class BinaryMarket:
    def __init__(self, initial_liquidity, initial_probability):
        self.pool_yes = initial_liquidity * initial_probability
        self.pool_no = initial_liquidity * (1 - initial_probability)
        self.p = 0.70  # CPMM parameter
        self.collected_fees = {"creator_fee": 0, "platform_fee": 0, "liquidity_fee": 0}

    def get_probability(self):
        return (self.p * self.pool_no) / ((1 - self.p) * self.pool_yes + self.p * self.pool_no)

    def calculate_shares(self, bet_amount, outcome):
        k = self.pool_yes ** self.p * self.pool_no ** (1 - self.p)
        if outcome == "YES":
            return self.pool_yes + bet_amount - (k * (bet_amount + self.pool_no) ** (self.p - 1)) ** (1 / self.p)
        else:
            return self.pool_no + bet_amount - (k * (bet_amount + self.pool_yes) ** -self.p) ** (1 / (1 - self.p))

    def get_fees(self, bet_amount, outcome):
        shares = self.calculate_shares(bet_amount, outcome)
        avg_prob = bet_amount / shares
        fee = 0.07 * avg_prob * (1 - avg_prob) * shares
        return {
            "creator_fee": fee * 0.5,
            "platform_fee": fee * 0.5,
            "liquidity_fee": 0
        }

    def place_bet(self, bet_amount, outcome):
        fees = self.get_fees(bet_amount, outcome)
        total_fees = sum(fees.values())
        remaining_bet = bet_amount - total_fees

        shares = self.calculate_shares(remaining_bet, outcome)

        if outcome == "YES":
            self.pool_yes -= shares - remaining_bet
            self.pool_no += remaining_bet
        else:
            self.pool_yes += remaining_bet
            self.pool_no -= shares - remaining_bet

        for fee_type, amount in fees.items():
            self.collected_fees[fee_type] += amount

        return shares

    def calculate_payout(self, shares, outcome, resolution):
        if resolution == "CANCEL":
            return shares
        elif resolution == outcome:
            return shares
        else:
            return 0

    def get_liquidity(self):
        return self.pool_yes ** self.p * self.pool_no ** (1 - self.p)

# Example usage
market = BinaryMarket(initial_liquidity=210, initial_probability=0.7)
print(f"Initial probability: {market.get_probability():.4f}")

bet_amount = 100
outcome = "YES"
shares = market.place_bet(bet_amount, outcome)
print(f"Shares received for {bet_amount} on {outcome}: {shares:.4f}")
print(f"New probability: {market.get_probability():.4f}")
print(f"Collected fees: {market.collected_fees}")

payout = market.calculate_payout(shares, outcome, "YES")
print(f"Payout if YES resolves: {payout:.4f}")


Initial probability: 0.5000
Shares received for 100 on YES: 146.2825
New probability: 0.7921
Collected fees: {'creator_fee': 1.1539995898498607, 'platform_fee': 1.1539995898498607, 'liquidity_fee': 0}
Payout if YES resolves: 146.2825


In [3]:
import math

class BinaryMarket:
    def __init__(self, pool_yes, pool_no, fee_percentage=0.02):
        self.pool_yes = pool_yes
        self.pool_no = pool_no
        self.fee_percentage = fee_percentage

    def get_probability(self):
        """Calculate probability based on the YES and NO pools."""
        return self.pool_yes / (self.pool_yes + self.pool_no)

    def get_log_odds(self, prob):
        """Calculate log-odds for the given probability."""
        return math.log(prob / (1 - prob))

    def place_bet(self, bet_direction, bet_size):
        """Place a bet in the specified direction (YES or NO) with the given bet size."""
        initial_prob = self.get_probability()
        initial_log_odds = self.get_log_odds(initial_prob)
        
        # Apply the bet to the respective pool
        if bet_direction == 'YES':
            self.pool_yes += bet_size
        elif bet_direction == 'NO':
            self.pool_no += bet_size
        else:
            raise ValueError("Bet direction must be 'YES' or 'NO'")
        
        # Calculate new probability after bet
        new_prob = self.get_probability()
        new_log_odds = self.get_log_odds(new_prob)
        
        # Max payout is based on the bet size and probability shift
        max_payout = bet_size * (1 / new_prob if bet_direction == 'YES' else 1 / (1 - new_prob))

        # Fee calculation
        fee = bet_size * self.fee_percentage

        # Liquidity pool changes
        pool_change_yes = self.pool_yes
        pool_change_no = self.pool_no

        return {
            'initial_probability': initial_prob,
            'new_probability': new_prob,
            'initial_log_odds': initial_log_odds,
            'new_log_odds': new_log_odds,
            'max_payout': max_payout,
            'fee': fee,
            'pool_change_yes': pool_change_yes,
            'pool_change_no': pool_change_no
        }

# Example usage
pool_yes = 910  # Initial YES shares in pool
pool_no = 1099   # Initial NO shares in pool

market = BinaryMarket(pool_yes, pool_no)

# User inputs
bet_direction = 'YES'  # Betting on YES
bet_size = 100  # Mana amount to bet

result = market.place_bet(bet_direction, bet_size)
result

{'initial_probability': 0.4529616724738676,
 'new_probability': 0.4788999525841631,
 'initial_log_odds': -0.18871135489272559,
 'new_log_odds': -0.08445034456831631,
 'max_payout': 208.81188118811883,
 'fee': 2.0,
 'pool_change_yes': 1010,
 'pool_change_no': 1099}

In [5]:
import math

class BinaryMarket:
    def __init__(self, pool_yes, pool_no, fee_percentage=0.02):
        self.pool_yes = pool_yes
        self.pool_no = pool_no
        self.fee_percentage = fee_percentage

    def get_probability(self):
        """Calculate probability based on the YES and NO pools."""
        return self.pool_yes / (self.pool_yes + self.pool_no)

    def get_log_odds(self, prob):
        """Calculate log-odds for the given probability."""
        return math.log(prob / (1 - prob))

    def place_bet(self, bet_direction, bet_size):
        """Place a bet in the specified direction (YES or NO) with the given bet size."""
        initial_prob = self.get_probability()
        initial_log_odds = self.get_log_odds(initial_prob)
        
        # Apply the bet to the respective pool
        if bet_direction == 'YES':
            self.pool_yes += bet_size
        elif bet_direction == 'NO':
            self.pool_no += bet_size
        else:
            raise ValueError("Bet direction must be 'YES' or 'NO'")
        
        # Calculate new probability after bet
        new_prob = self.get_probability()
        new_log_odds = self.get_log_odds(new_prob)
        
        # Max payout is based on the bet size and probability shift
        max_payout = bet_size * (1 / new_prob if bet_direction == 'YES' else 1 / (1 - new_prob))

        # Fee calculation
        fee = bet_size * self.fee_percentage

        # Liquidity pool changes
        pool_change_yes = self.pool_yes
        pool_change_no = self.pool_no

        return {
            'initial_probability': initial_prob,
            'new_probability': new_prob,
            'initial_log_odds': initial_log_odds,
            'new_log_odds': new_log_odds,
            'max_payout': max_payout,
            'fee': fee,
            'pool_change_yes': pool_change_yes,
            'pool_change_no': pool_change_no
        }

# Example usage
pool_yes = 910  # Initial YES shares in pool
pool_no = 1099   # Initial NO shares in pool

market = BinaryMarket(pool_yes, pool_no)

# User inputs
bet_direction = 'YES'  # Betting on YES
bet_size = 50  # Mana amount to bet

result = market.place_bet(bet_direction, bet_size)
print(result)


{'initial_probability': 0.4529616724738676, 'new_probability': 0.4662457503642545, 'initial_log_odds': -0.18871135489272559, 'new_log_odds': -0.13522266994173945, 'max_payout': 107.23958333333334, 'fee': 1.0, 'pool_change_yes': 960, 'pool_change_no': 1099}


In [9]:
import math

class Maniswap:
    def __init__(self, pool_yes, pool_no, initial_p=0.5, fee_percentage=0.02):
        self.pool_yes = pool_yes
        self.pool_no = pool_no
        self.p = initial_p  # Adjustable parameter for initial probability
        self.k = (pool_yes ** self.p) * (pool_no ** (1 - self.p))  # Generalized constant
        self.fee_percentage = fee_percentage

    def get_probability(self):
        """Calculate market probability based on YES and NO pools using Maniswap formula."""
        return (self.p * self.pool_no) / ((self.p * self.pool_no) + (1 - self.p) * self.pool_yes)

    def get_log_odds(self, prob):
        """Calculate log-odds for the given probability."""
        return math.log(prob / (1 - prob))

    def place_bet(self, bet_direction, bet_size):
        """Place a bet in the specified direction (YES or NO) and adjust the pools accordingly."""
        initial_prob = self.get_probability()
        initial_log_odds = self.get_log_odds(initial_prob)

        # Apply the bet and adjust pools accordingly
        if bet_direction == 'YES':
            self.pool_yes += bet_size
        elif bet_direction == 'NO':
            self.pool_no += bet_size
        else:
            raise ValueError("Bet direction must be 'YES' or 'NO'")
        
        # Recalculate the constant k to maintain the market's balance
        self.k = (self.pool_yes ** self.p) * (self.pool_no ** (1 - self.p))
        
        # Calculate new probability and log-odds
        new_prob = self.get_probability()
        new_log_odds = self.get_log_odds(new_prob)
        
        # Max payout based on the bet and new probability
        max_payout = bet_size * (1 / new_prob if bet_direction == 'YES' else 1 / (1 - new_prob))

        # Calculate fee
        fee = bet_size * self.fee_percentage

        return {
            'initial_probability': initial_prob,
            'new_probability': new_prob,
            'initial_log_odds': initial_log_odds,
            'new_log_odds': new_log_odds,
            'max_payout': max_payout,
            'fee': fee,
            'updated_pool_yes': self.pool_yes,
            'updated_pool_no': self.pool_no
        }

    def inject_liquidity(self, amount):
        """Inject liquidity into the market while maintaining the current probability."""
        # Adding YES and NO shares proportional to the existing pool
        self.pool_yes += amount
        self.pool_no += amount

        # Recalculate k based on the new pools
        self.k = (self.pool_yes ** self.p) * (self.pool_no ** (1 - self.p))

        return {
            'updated_pool_yes': self.pool_yes,
            'updated_pool_no': self.pool_no,
            'new_k': self.k
        }

# Example usage
pool_yes = 910  # Initial YES shares in pool
pool_no = 1099   # Initial NO shares in pool
initial_p = 0.50  # Starting the market at a 33% probability

market = Maniswap(pool_yes, pool_no, initial_p)

# Example: A trader places a bet on YES with 500 mana
bet_direction = 'YES'
bet_size = 50

result = market.place_bet(bet_direction, bet_size)
print(result)

# Injecting liquidity into the market
liquidity_result = market.inject_liquidity(1000)
print(liquidity_result)


{'initial_probability': 0.5470383275261324, 'new_probability': 0.5337542496357455, 'initial_log_odds': 0.1887113548927254, 'new_log_odds': 0.13522266994173923, 'max_payout': 93.67606915377617, 'fee': 1.0, 'updated_pool_yes': 960, 'updated_pool_no': 1099}
{'updated_pool_yes': 1960, 'updated_pool_no': 2099, 'new_k': 2028.3096410558223}


In [11]:
import math
from typing import Dict, Tuple, List

# Constants from fees.ts
TAKER_FEE_CONSTANT = 0.07
CREATORS_EARN_WHOLE_FEE_UP_TO = 1000
TWOMBA_ENABLED = False  # Assuming this is False for this implementation

class Fees:
    def __init__(self, creator_fee: float = 0, platform_fee: float = 0, liquidity_fee: float = 0):
        self.creator_fee = creator_fee
        self.platform_fee = platform_fee
        self.liquidity_fee = liquidity_fee

class CpmmState:
    def __init__(self, pool: Dict[str, float], p: float, collected_fees: Fees):
        self.pool = pool
        self.p = p
        self.collected_fees = collected_fees

def get_taker_fee(shares: float, prob: float) -> float:
    return TAKER_FEE_CONSTANT * prob * (1 - prob) * shares

def get_fees_split(total_fees: float, previously_collected_fees: Fees) -> Fees:
    if TWOMBA_ENABLED:
        return Fees(creator_fee=0, platform_fee=total_fees, liquidity_fee=0)

    before_1k = max(0, CREATORS_EARN_WHOLE_FEE_UP_TO - previously_collected_fees.creator_fee)
    all_to_creator_amount = min(total_fees, before_1k)
    split_with_creator_amount = total_fees - all_to_creator_amount
    
    return Fees(
        creator_fee=all_to_creator_amount + split_with_creator_amount * 0.5,
        platform_fee=split_with_creator_amount * 0.5,
        liquidity_fee=0
    )

def get_cpmm_probability(pool: Dict[str, float], p: float) -> float:
    yes, no = pool['YES'], pool['NO']
    return (p * no) / ((1 - p) * yes + p * no)

def calculate_cpmm_shares(pool: Dict[str, float], p: float, bet_amount: float, bet_choice: str) -> float:
    if bet_amount == 0:
        return 0

    y, n = pool['YES'], pool['NO']
    k = y ** p * n ** (1 - p)

    if bet_choice == 'YES':
        return y + bet_amount - (k * (bet_amount + n) ** (p - 1)) ** (1 / p)
    else:
        return n + bet_amount - (k * (bet_amount + y) ** -p) ** (1 / (1 - p))

def get_cpmm_fees(state: CpmmState, bet_amount: float, outcome: str) -> Tuple[float, float, Fees]:
    fee = 0
    for _ in range(10):
        bet_amount_after_fee = bet_amount - fee
        shares = calculate_cpmm_shares(state.pool, state.p, bet_amount_after_fee, outcome)
        average_prob = bet_amount_after_fee / shares
        fee = get_taker_fee(shares, average_prob)

    total_fees = fee if bet_amount != 0 else 0
    fees = get_fees_split(total_fees, state.collected_fees)
    remaining_bet = bet_amount - total_fees

    return remaining_bet, total_fees, fees

def calculate_cpmm_purchase(state: CpmmState, bet: float, outcome: str, free_fees: bool = False) -> Tuple[float, Dict[str, float], float, Fees]:
    if free_fees:
        remaining_bet, fees = bet, Fees()
    else:
        remaining_bet, _, fees = get_cpmm_fees(state, bet, outcome)

    shares = calculate_cpmm_shares(state.pool, state.p, remaining_bet, outcome)
    y, n = state.pool['YES'], state.pool['NO']
    fee = fees.liquidity_fee

    if outcome == 'YES':
        new_y, new_n = y - shares + remaining_bet + fee, n + remaining_bet + fee
    else:
        new_y, new_n = y + remaining_bet + fee, n - shares + remaining_bet + fee

    post_bet_pool = {'YES': new_y, 'NO': new_n}
    new_pool, new_p = add_cpmm_liquidity(post_bet_pool, state.p, fee)

    return shares, new_pool, new_p, fees

def add_cpmm_liquidity(pool: Dict[str, float], p: float, amount: float) -> Tuple[Dict[str, float], float]:
    prob = get_cpmm_probability(pool, p)
    y, n = pool['YES'], pool['NO']
    
    numerator = prob * (amount + y)
    denominator = amount - n * (prob - 1) + prob * y
    new_p = numerator / denominator

    new_pool = {'YES': y + amount, 'NO': n + amount}

    old_liquidity = get_cpmm_liquidity(pool, new_p)
    new_liquidity = get_cpmm_liquidity(new_pool, new_p)
    liquidity = new_liquidity - old_liquidity

    return new_pool, new_p

def get_cpmm_liquidity(pool: Dict[str, float], p: float) -> float:
    return pool['YES'] ** p * pool['NO'] ** (1 - p)

# Additional helper functions can be implemented as needed

# Example usage:
if __name__ == "__main__":
    initial_pool = {'YES': 910, 'NO': 1099}
    initial_state = CpmmState(initial_pool, 0.50, Fees())
    
    bet_amount = 50
    outcome = 'YES'
    
    shares, new_pool, new_p, fees = calculate_cpmm_purchase(initial_state, bet_amount, outcome)
    
    print(f"Shares: {shares}")
    print(f"New pool: {new_pool}")
    print(f"New p: {new_p}")
    print(f"Fees: Creator: {fees.creator_fee}, Platform: {fees.platform_fee}, Liquidity: {fees.liquidity_fee}")

Shares: 86.959217394382
New pool: {'YES': 871.5392729569512, 'NO': 1147.4984903513332}
New p: 0.5
Fees: Creator: 1.5015096486668114, Platform: 0.0, Liquidity: 0


In [15]:
import math
from typing import Dict, Tuple, List

# Constants from fees.ts
TAKER_FEE_CONSTANT = 0.07
CREATORS_EARN_WHOLE_FEE_UP_TO = 1000
TWOMBA_ENABLED = False  # Assuming this is False for this implementation

class Fees:
    def __init__(self, creator_fee: float = 0, platform_fee: float = 0, liquidity_fee: float = 0):
        self.creator_fee = creator_fee
        self.platform_fee = platform_fee
        self.liquidity_fee = liquidity_fee

class CpmmState:
    def __init__(self, pool: Dict[str, float], p: float, collected_fees: Fees):
        self.pool = pool
        self.p = p
        self.collected_fees = collected_fees

def get_taker_fee(shares: float, prob: float) -> float:
    return TAKER_FEE_CONSTANT * prob * (1 - prob) * shares

def get_fees_split(total_fees: float, previously_collected_fees: Fees) -> Fees:
    if TWOMBA_ENABLED:
        return Fees(creator_fee=0, platform_fee=total_fees, liquidity_fee=0)

    before_1k = max(0, CREATORS_EARN_WHOLE_FEE_UP_TO - previously_collected_fees.creator_fee)
    all_to_creator_amount = min(total_fees, before_1k)
    split_with_creator_amount = total_fees - all_to_creator_amount
    
    return Fees(
        creator_fee=all_to_creator_amount + split_with_creator_amount * 0.5,
        platform_fee=split_with_creator_amount * 0.5,
        liquidity_fee=0
    )

def get_cpmm_probability(pool: Dict[str, float], p: float) -> float:
    yes, no = pool['YES'], pool['NO']
    return (p * no) / ((1 - p) * yes + p * no)

def calculate_cpmm_shares(pool: Dict[str, float], p: float, bet_amount: float, bet_choice: str) -> float:
    if bet_amount == 0:
        return 0

    y, n = pool['YES'], pool['NO']
    k = y ** p * n ** (1 - p)

    if bet_choice == 'YES':
        return y + bet_amount - (k * (bet_amount + n) ** (p - 1)) ** (1 / p)
    else:
        return n + bet_amount - (k * (bet_amount + y) ** -p) ** (1 / (1 - p))

def get_cpmm_fees(state: CpmmState, bet_amount: float, outcome: str) -> Tuple[float, float, Fees]:
    fee = 0
    for _ in range(10):
        bet_amount_after_fee = bet_amount - fee
        shares = calculate_cpmm_shares(state.pool, state.p, bet_amount_after_fee, outcome)
        average_prob = bet_amount_after_fee / shares
        fee = get_taker_fee(shares, average_prob)

    total_fees = fee if bet_amount != 0 else 0
    fees = get_fees_split(total_fees, state.collected_fees)
    remaining_bet = bet_amount - total_fees

    return remaining_bet, total_fees, fees

def calculate_cpmm_purchase(state: CpmmState, bet: float, outcome: str, free_fees: bool = False) -> Tuple[float, Dict[str, float], float, Fees]:
    if free_fees:
        remaining_bet, fees = bet, Fees()
    else:
        remaining_bet, _, fees = get_cpmm_fees(state, bet, outcome)

    shares = calculate_cpmm_shares(state.pool, state.p, remaining_bet, outcome)
    y, n = state.pool['YES'], state.pool['NO']
    fee = fees.liquidity_fee

    if outcome == 'YES':
        new_y, new_n = y - shares + remaining_bet + fee, n + remaining_bet + fee
    else:
        new_y, new_n = y + remaining_bet + fee, n - shares + remaining_bet + fee

    post_bet_pool = {'YES': new_y, 'NO': new_n}
    new_pool, _, new_p = add_cpmm_liquidity(post_bet_pool, state.p, fee)

    return shares, new_pool, new_p, fees

def add_cpmm_liquidity(pool: Dict[str, float], p: float, amount: float) -> Tuple[Dict[str, float], float, float]:
    prob = get_cpmm_probability(pool, p)
    y, n = pool['YES'], pool['NO']
    
    numerator = prob * (amount + y)
    denominator = amount - n * (prob - 1) + prob * y
    new_p = numerator / denominator

    new_pool = {'YES': y + amount, 'NO': n + amount}

    old_liquidity = get_cpmm_liquidity(pool, p)
    new_liquidity = get_cpmm_liquidity(new_pool, new_p)
    liquidity = new_liquidity - old_liquidity

    return new_pool, liquidity, new_p

def get_cpmm_liquidity(pool: Dict[str, float], p: float) -> float:
    return pool['YES'] ** p * pool['NO'] ** (1 - p)

# Additional helper functions can be implemented as needed

# Example usage:
if __name__ == "__main__":
    initial_pool = {'YES': 910, 'NO': 1099}
    initial_state = CpmmState(initial_pool, 0.55, Fees())
    
    bet_amount = 50
    outcome = 'YES'
    
    shares, new_pool, new_p, fees = calculate_cpmm_purchase(initial_state, bet_amount, outcome)
    
    print(f"Shares: {shares}")
    print(f"New pool: {new_pool}")
    print(f"New p: {new_p}")
    print(f"Fees: Creator: {fees.creator_fee}, Platform: {fees.platform_fee}, Liquidity: {fees.liquidity_fee}")

Shares: 80.34650863577087
New pool: {'YES': 878.3101276849291, 'NO': 1147.6566363206998}
New p: 0.55
Fees: Creator: 1.343363679300114, Platform: 0.0, Liquidity: 0
